In [11]:
from dotenv import load_dotenv
import os

load_dotenv()  # this will load variables from .env

# Now you can access the variables
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')


In [12]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", temperature=0)

text = "What would be a good company name for a company that makes colorful socks?"

print(llm(text))




Rainbow Socks Co.


In [5]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 48
	Prompt Tokens: 4
	Completion Tokens: 44
Successful Requests: 1
Total Cost (USD): $0.00096


In [6]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [8]:
from langchain import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

' Life is about finding joy and contentment in whatever you do, and living each day to the fullest!'

In [9]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [13]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


paris


In [14]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)

In [16]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

'Paris\nBlue Whale\nNitrogen\nYellow'

In [19]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/lib/python3.9/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/lib/python3.9/site-packages/langchain/llms/openai.py:748: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [20]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

In [22]:
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)
print(summarized_text)

LangChain offers various modules for building language model applications, which can be used individually or combined to create complex applications, with the most basic building block being calling an LLM on input, as demonstrated in a simple example of building a company name generator.


In [23]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

In [25]:
source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
print(translated_text)

Votre texte ici


In [33]:
import xml.etree.ElementTree as ET

def extract_urls_from_xml_file(file_path):
    try:
        # Parse the XML file
        tree = ET.parse(file_path)

        # Get the root of the XML document
        root = tree.getroot()

        # Define the namespace
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        # Extract URLs
        urls = [element.text for element in root.findall('sitemap:loc', ns)]

        # Print the URLs
        for url in urls:
            print(url)
    except ET.ParseError as e:
        print(f'Parse error: {e}')

# Specify the path to your XML file
file_path = '/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/dune_xml.txt'

extract_urls_from_xml_file(file_path)






In [34]:
import xml.etree.ElementTree as ET

def extract_urls_from_xml_file(file_path, output_file_path):
    try:
        # Parse the XML file
        tree = ET.parse(file_path)

        # Get the root of the XML document
        root = tree.getroot()

        # Define the namespace
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        # Extract URLs
        urls = [element.text for element in root.findall('sitemap:url', ns)]
        
        # Create a new root for the output XML
        new_root = ET.Element('urlset')
        new_root.set('xmlns', 'http://www.sitemaps.org/schemas/sitemap/0.9')

        # For each URL, create a new url element with a loc subelement
        for url in urls:
            url_element = ET.SubElement(new_root, 'url')
            loc_element = ET.SubElement(url_element, 'loc')
            loc_element.text = url

        # Wrap the root element in an ElementTree and write it to the output file
        new_tree = ET.ElementTree(new_root)
        new_tree.write(output_file_path, encoding='utf-8', xml_declaration=True)

    except ET.ParseError as e:
        print(f'Parse error: {e}')

# Specify the path to your XML file and the output file
file_path = '/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/dune_xml.txt'
output_file_path = '/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/dune_xml.xml'

extract_urls_from_xml_file(file_path, output_file_path)


In [35]:
import xml.etree.ElementTree as ET

def extract_urls_from_xml_file(file_path, output_file_path):
    try:
        # Parse the XML file
        tree = ET.parse(file_path)

        # Get the root of the XML document
        root = tree.getroot()

        # Define the namespace
        ns = {'sitemap': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

        # Extract URLs
        urls = [element.text for element in root.findall('./sitemap:url/sitemap:loc', ns)]

        # Create a new root for the output XML
        new_root = ET.Element('urlset')
        new_root.set('xmlns', 'http://www.sitemaps.org/schemas/sitemap/0.9')

        # For each URL, create a new url element with a loc subelement
        for url in urls:
            url_element = ET.SubElement(new_root, 'url')
            loc_element = ET.SubElement(url_element, 'loc')
            loc_element.text = url

        # Wrap the root element in an ElementTree and write it to the output file
        new_tree = ET.ElementTree(new_root)
        new_tree.write(output_file_path, encoding='utf-8', xml_declaration=True)

    except ET.ParseError as e:
        print(f'Parse error: {e}')

# Specify the path to your XML file and the output file
file_path = '/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/dune_xml.txt'
output_file_path = '/Users/dannyhughes/Documents/activeloop_course/activeloop_venv/dune_xml.xml'

extract_urls_from_xml_file(file_path, output_file_path)


In [40]:
import os
import xml.etree.ElementTree as ET

# Set the local path to the cloned repository
repo_path = "/Users/dannyhughes/Documents/spellbook"

# Set the GitHub repository URL
github_repo_url = "https://github.com/duneanalytics/spellbook"

def add_files_to_sitemap(root, path):
    # Create the full path to the current directory or file
    full_path = os.path.join(repo_path, path)

    if os.path.isdir(full_path):
        # If it's a directory, iterate over its contents
        for file_name in os.listdir(full_path):
            # Call this function recursively for each file in the directory
            add_files_to_sitemap(root, os.path.join(path, file_name))
    else:
        # This is a file, add its URL to the XML
        # Construct the GitHub file URL based on the repository URL and the relative file path
        file_url = f"{github_repo_url}/blob/main/{path}"

        # Create a new "url" XML element
        url_elem = ET.SubElement(root, "url")

        # Create a "loc" element containing the file URL
        loc_elem = ET.SubElement(url_elem, "loc")
        loc_elem.text = file_url

# Create a new XML root element
root = ET.Element("urlset", xmlns="http://www.sitemaps.org/schemas/sitemap/0.9")

# Start the recursion with the root directory
add_files_to_sitemap(root, "models")

# Convert the XML to a string
xml_str = ET.tostring(root, encoding="unicode")

# Save the XML to a file
with open("github_sitemap.xml", "w") as f:
    f.write(xml_str)